In [5]:
import math
import pandas as pd
import numpy
import yfinance as yf


In [6]:
# Downloading the Data
data = yf.download("GOOGL", start="2010-01-01", end="2023-05-01", interval="1d")

[*********************100%***********************]  1 of 1 completed


In [7]:
# Calculating Daily Returns 
# Assuming Initial Price to be Rs. 100

initial_price = 100

start_price = []
end_price = []
max_price = []
min_price = []
daily_returns = []

for i, date in enumerate(data.index):
    if i == 0:
        start_price.append(initial_price)
    else:
        start_price.append(end_price[i-1])

    end_price.append((start_price[i] * data.Close[date]) / data.Open[date])
    max_price.append((start_price[i] * data.High[date]) / data.Open[date])
    min_price.append((start_price[i] * data.Low[date]) / data.Open[date])
    daily_returns.append((end_price[i] - start_price[i]) / start_price[i])

data["Start"] = start_price
data["End"] = end_price
data["Min"] = min_price
data["Max"] = max_price
data["Returns"] = daily_returns  

In [8]:
# Calculating Cumulative Returns

cumulative_returns = (data["End"][-1] - initial_price)/initial_price
cumulative_returns

-0.03504922389806154

In [9]:
# Calculating Maximum Drawdown

maximum_drawdown = 0
MAX = -math.inf
MIN = math.inf

for date in data.index:
    if data["Max"][date] > MAX:
        MAX = data["Max"][date]
        MIN = math.inf
    if data["Min"][date] < MIN:
        MIN = data["Min"][date]

    maximum_drawdown = min(maximum_drawdown, ((MIN - MAX)/MAX))

maximum_drawdown

-0.5781311425986818

In [10]:
# Volitality 
# It is defined as the annualised standard deviation of returns over a period of time

StdDevx = data.Returns.std()

volitality = StdDevx * math.sqrt(252)  # 252 Trading Days in an Year
volitality

0.2050625531209582

In [11]:
# Sharpe & Sortino Ratio 

Rx = data.Returns.mean() * 252                                          # Annualized Returns 
Volx = data.Returns.std() * math.sqrt(252)                              # Volitality or Annualized StdDev
StdDev_Neg = data.Returns[data.Returns < 0].std() * math.sqrt(252)      # Annualized DownSide StdDev
Rf = 0.06                                                               # RISK FREE RATE -> Based on Indian Govt. 10 Year Bond Rate

sharpe_ratio = ((Rx - Rf)/Volx)
sortino_ratio = ((Rx - Rf)/StdDev_Neg)
print(sharpe_ratio, sortino_ratio)

-0.20294590790609363 -0.28552140023083


In [12]:
def get_stock_data(ticker, start_date="2010-01-01", end_date="2023-05-01"):
    # Downloading the Data
    data = yf.download(ticker, start=start_date, end=end_date, interval="1d", progress=False)

    # Calculating Daily Returns 
    # Assuming Initial Price to be Rs. 100
    initial_price = 100

    start_price = []
    end_price = []
    max_price = []
    min_price = []
    daily_returns = []

    for i, date in enumerate(data.index):
        if i == 0:
            start_price.append(initial_price)
        else:
            start_price.append(end_price[i-1])

        end_price.append((start_price[i] * data.Close[date]) / data.Open[date])
        max_price.append((start_price[i] * data.High[date]) / data.Open[date])
        min_price.append((start_price[i] * data.Low[date]) / data.Open[date])
        daily_returns.append((end_price[i] - start_price[i]) / start_price[i])

    data["Start"] = start_price
    data["End"] = end_price
    data["Min"] = min_price
    data["Max"] = max_price
    data["Returns"] = daily_returns  

    # Calculating Cumulative Returns
    cumulative_returns = (data["End"][-1] - initial_price)/initial_price

    # Calculating Maximum Drawdown
    maximum_drawdown = 0
    MAX = -math.inf
    MIN = math.inf


    for date in data.index:
        if data["Max"][date] > MAX:
            MAX = data["Max"][date]
            MIN = math.inf
        if data["Min"][date] < MIN:
            MIN = data["Min"][date]

        maximum_drawdown = min(maximum_drawdown, ((MIN - MAX)/MAX))

    # Volitality 
    # It is defined as the annualised standard deviation of returns over a period of time
    StdDevx = data.Returns.std()
    volitality = StdDevx * math.sqrt(252)  # 252 Trading Days in an Year

    # Sharpe & Sortino Ratio 
    Rx = data.Returns.mean() * 252                                          # Annualized Returns 
    Volx = data.Returns.std() * math.sqrt(252)                              # Volitality or Annualized StdDev
    StdDev_Neg = data.Returns[data.Returns < 0].std() * math.sqrt(252)      # Annualized DownSide StdDev
    Rf = 0.06                                                               # RISK FREE RATE -> Based on Indian Govt. 10 Year Bond Rate

    sharpe_ratio = ((Rx - Rf)/Volx)
    sortino_ratio = ((Rx - Rf)/StdDev_Neg)


    return cumulative_returns*100, volitality*100, sharpe_ratio, sortino_ratio, maximum_drawdown*100
    

In [15]:
indices = ['^NSEI', '^RUT', '^STI', '^GSPC', '^VIX']
equities = ['GOOGL', 'SBIN.BO', 'AMZN', 'GOLDBEES.BO', 'ASIANPAINT.BO']

print("Indices Data -")
for indx in indices: 
    print('Index: {} ({}), Cumulative Returns: {:.2f} %, Volitality: {:.2f} %, Sharpe Ratio: {:.3f}, Sortino Ratio: {:.3f}, Maximum Drawdown: {:.2f} %'.format(indx, yf.Ticker(indx).info['longName'], *get_stock_data(indx)))
print("\nEquity Data -")
for eq in equities: 
    print('Stock: {}, Cumulative Returns: {:.2f}%, Volatility: {:.2f}%, Sharpe Ratio: {:.3f}, Sortino Ratio: {}'.format(stock_ticker, cumulative_returns * 100, volatility * 100, sharpe_ratio, sortino_ratio))


Indices Data -
Index: ^NSEI (NIFTY 50), Cumulative Returns: -90.67 %, Volitality: 14.54 %, Sharpe Ratio: -1.598, Sortino Ratio: -2.272, Maximum Drawdown: -92.21 %
Index: ^RUT (Russell 2000), Cumulative Returns: 81.88 %, Volitality: 20.34 %, Sharpe Ratio: 0.028, Sortino Ratio: 0.038, Maximum Drawdown: -45.70 %
Index: ^STI (STI Index), Cumulative Returns: -41.63 %, Volitality: 10.94 %, Sharpe Ratio: -0.866, Sortino Ratio: -1.196, Maximum Drawdown: -50.59 %
Index: ^GSPC (S&P 500), Cumulative Returns: 122.67 %, Volitality: 14.86 %, Sharpe Ratio: 0.076, Sortino Ratio: 0.096, Maximum Drawdown: -23.72 %
Index: ^VIX (CBOE Volatility Index), Cumulative Returns: -100.00 %, Volitality: 109.89 %, Sharpe Ratio: -1.228, Sortino Ratio: -2.496, Maximum Drawdown: -100.00 %

Equity Data -


NameError: ignored